In [1]:
import numpy as np
import cv2

In [2]:
def rectify(corner_points):
    corner_points = corner_points.reshape((4, 2))
    new_points = np.zeros((4, 2), dtype=np.float32)
    temp1 = corner_points.sum(1)
    new_points[0] = corner_points[np.argmin(temp1)]
    new_points[2] = corner_points[np.argmax(temp1)]
    temp2 = np.diff(corner_points, axis=1)
    new_points[1] = corner_points[np.argmin(temp2)]
    new_points[3] = corner_points[np.argmax(temp2)]
    return new_points

In [3]:
# returns a [4,2] array containig the four corner points of the rectangle by accepting all the contours found in the image

def get_rectangle(contours):
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    for contour in contours:
        p = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * p, True)

        if len(approx) == 4:
            return approx

In [4]:
#shows the different images formed in the process

def show_images(scanned_image, contour_image, edged_image):
    cv2.imshow("Initial image", contour_image)                                # the actual image
    cv2.imshow("Scanned output, colored", scanned_image)                      # the scanned output, for birds eye view
    cv2.imshow("Edged image", edged_image)                                    # shows the canny image
    grey_image = cv2.cvtColor(scanned_image, cv2.COLOR_BGR2GRAY)
    _, binary_image = cv2.threshold(grey_image, 150, 255, cv2.THRESH_BINARY)
    cv2.imshow("Binary output", binary_image)                                 # shows the binary output image
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [5]:
# the functions that calls in the respective functions and does the main task

def get_scanned_image(image, width, height):
    image = cv2.resize(image, (width, height))
    grey_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(grey_image, (5, 5), 0)                             # applying gaussian blur
    edged_image = cv2.Canny(blurred_image, 0, 50)                                       # image having the detected edges
    contours, _ = cv2.findContours(edged_image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE) # find the contours of the images
    rectangle = get_rectangle(contours)                                                 # gets the corner points of the document
    approx_rectangle = rectify(rectangle)
    dimensions = np.float32([[0, 0], [width, 0], [width, height], [0, height]])
    matrix = cv2.getPerspectiveTransform(approx_rectangle, dimensions)
    scanned_image = cv2.warpPerspective(image, matrix, (width, height))                 # gets the birds-eye view of the document
    cv2.drawContours(image, [rectangle], -1, (0, 255, 0), 3)
    return scanned_image, image, edged_image

In [7]:
img = cv2.imread('doc4.jpeg')
scan, c_image, e_image = get_scanned_image(img, 600, 800)
show_images(scan, c_image, e_image)